In [11]:
path = "/home/efectn/python-music-classification"

In [12]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model(path + "/models/model_dnn.h5")


In [27]:
import librosa
import math

n_fft=2048
hop_length=512
n_mfcc=13

genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

audio, fs = librosa.load(path+"/blues.00002.wav", sr=22500)

duration = librosa.get_duration(y=audio, sr=fs)

print(f"Dosya süresi: {duration:.2f} saniye")

num_segments = int(duration / 2)
samples_per_track = fs * duration
samps_per_segment = int(samples_per_track/num_segments)
mfccs_per_segment = math.ceil(samps_per_segment/hop_length)

mfccs = []
expected_frames = 88

for seg in range(num_segments):
    start_sample = seg * samps_per_segment
    end_sample = start_sample + samps_per_segment

    mfcc = librosa.feature.mfcc(y=audio[start_sample:end_sample], sr=fs, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
    mfcc = mfcc.T

    if mfcc.shape[0] > expected_frames:
        mfcc = mfcc[:expected_frames, :]
    elif mfcc.shape[0] < expected_frames:
        pad_width = expected_frames - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')

    mfccs.append(mfcc)
        
X_train_cnn = np.array(mfccs)

def make_prediction(model, X):
    global genres
    preds_probs = []

    for X_current in X:
        X_current = X_current[np.newaxis, ...]  
        pred = model.predict(X_current, verbose=0)[0]
        preds_probs.append(pred)

    preds_probs = np.array(preds_probs)
    avg_probs = np.mean(preds_probs, axis=0)

    pred_idx = np.argmax(avg_probs)
    pred_genre = genres[pred_idx]
    avg_probs_percent = avg_probs * 100

    return pred_genre, pred_idx, avg_probs_percent


genre_name, genre_index, avg_probs = make_prediction(model, X_train_cnn)

print(f"\nGenel Tahmin: {genre_name.upper()} (Sınıf ID: {genre_index})")
print("Ortalama Olasılıklar:")
for i, genre in enumerate(genres):
    print(f"  {genre:10s}: {avg_probs[i]:5.2f}%")

Dosya süresi: 30.01 saniye

Genel Tahmin: BLUES (Sınıf ID: 0)
Ortalama Olasılıklar:
  blues     : 51.53%
  classical :  0.13%
  country   : 12.98%
  disco     :  2.95%
  hiphop    :  1.05%
  jazz      :  0.40%
  metal     :  0.68%
  pop       :  1.36%
  reggae    : 12.32%
  rock      : 16.59%
